In [ ]:
import pandas as pd
import numpy as np
# import random as rd

import matplotlib.pyplot as plt
# import seaborn as sns

In [ ]:
def points_generator(n_tests):
    azimuth = np.random.uniform(min = 0, max = 2*np.pi(), size = (n_tests,1))
    polar = np.random.uniform(min = 0, max = np.pi(), size = (n_tests,1))
    coordinates = np.vstack((azimuth, polar))
    return coordinates


In [4]:
points_generator(10)

NameError: name 'pi' is not defined